In [1]:
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import sklearn


sample_submission.csv - A submission file in the correct format.
PassengerId - Id for each passenger in the test set.
Transported - The target. For each passenger, predict either True or False.


In [2]:
sample_submission_df = pl.read_csv('sample_submission.csv')
train_df = pl.read_parquet('train_df.parquet')
test_df = pl.read_parquet('test_df.parquet')

In [3]:
train_df = train_df.drop_nulls()
y = train_df['Transported']
X = train_df.drop('Transported')

y_test = test_df['Transported']
X_test = test_df.drop('Transported')

In [23]:
classifier = sklearn.ensemble.HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_iter=100,
    max_leaf_nodes=None,
    max_depth=None,
    min_samples_leaf=32,
    l2_regularization=2.0,
    max_features=0.8,
    categorical_features="from_dtype",
    warm_start=True,
    scoring="f1",
    validation_fraction=0.25,
    verbose=1,
    random_state=42,
)
classifier.fit(X, y)
score = classifier.score(X, y)
print('Score:', score)

Binning 0.001 GB of training data: 0.014 s
Fitting gradient boosted rounds:
Fit 100 trees in 1.351 s, (15161 total leaves)
Time spent computing histograms: 0.238s
Time spent finding best splits:  0.316s
Time spent applying splits:      0.325s
Time spent predicting:           0.010s
Score: 0.9189117802430006


In [24]:
classifier.fit(X, y)
score = classifier.score(X, y)
print('Score:', score)

Binning 0.001 GB of training data: 0.015 s
Fitting gradient boosted rounds:
Fit 100 trees in 0.052 s, (15161 total leaves)
Time spent computing histograms: 0.000s
Time spent finding best splits:  0.000s
Time spent applying splits:      0.000s
Time spent predicting:           0.000s
Score: 0.9189117802430006


In [25]:
pred_proba = classifier.predict_proba(X_test)
preds = classifier.predict(X_test)


In [26]:
test_csv = pl.read_csv('test.csv')
transported = pl.Series('Transported', preds).cast(pl.String).str.to_titlecase()
submission_df = test_csv.select('PassengerId').hstack(pl.DataFrame(transported))
submission_df.write_csv('spaceship-titanic-submission.csv')


In [27]:
!kaggle competitions submit -c spaceship-titanic -f spaceship-titanic-submission.csv -m "spaceship-titanic-submission.csv"

100%|██████████████████████████████████████| 56.6k/56.6k [00:01<00:00, 47.6kB/s]
Successfully submitted to Spaceship Titanic